<a href="https://colab.research.google.com/github/Leia2000/Bachelorarbeit_Skripte/blob/main/create_mask_and_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# remove all open variables and files
rm(list = ls())

# load necessaries packages
library(terra)
library(sf)

# !!!!! load the paths to all image-subsets overlapping with the
# reference polygons
fils <- list.files("C:/Users/hanne/OneDrive/Desktop/BThesis/tif/", pattern=".tif$", full.names = T)

# !!!!! load the reference polygons
tree <- vect("C:/Users/hanne/Documents/deadwood.shp")
## plot tree shapefile to see of it has been loaded correctly (optional)
plot(tree)

# start a loop that loads one image-subset after the other
for (u in 1:length(fils)){

  # import image-subset
  parakou <- rast(fils[u])
  # plot image to see of it has been loaded correctly (optional)
  plot(parakou)

  # crop the extent of the file containing the reference polygons
  # to the extent of the current image subset
  tree2 = crop(tree, ext(parakou))

  # obtain the pixel size of the current image subset
  res_ras <- res(parakou)[1]

  # get the geographic extent (xmin, xmas, ymin, ymax) of the
  # current image subset
  studarea <- ext(parakou)

  # !!!!! define the tile size
  # the tile size will define how large the tiles you will feed into
  # the deep learning algorithm are. The tile size does have an effect on
  # the way the algorithm learns and is hence one of the parameters you can
  # adjust. the larger the value you chose, the larger the tiles will be
  # and the less tiles you will create. In this example the tile size would be
  # 128 by 128 pixels
  tilesize = 256*res_ras

  # create the corner-coordinates of the tiles
  xsteps <- seq(studarea[1], studarea[2], tilesize)
  ysteps <- seq(studarea[3], studarea[4], tilesize)

  # start a double loop to clip the current sub-image into
  # equally sized, non-overlapping tiles
  for (i1 in 1:(length(xsteps)-1)){
    for (i2 in 1:(length(ysteps)-1)){

      # compose the extent of the current tile
      clipext <- ext(xsteps[i1], xsteps[i1+1], ysteps[i2], ysteps[i2+1])

      # crop the image to the current tile extent
      img <- crop(parakou, clipext)
      # rearrange the order of the channels
      img2 <- c(img[[3]],img[[2]],img[[1]])

      # crop the reference polygon file to the extent of current tile
      mask_dummy <- crop(tree, clipext)

      # check if the cropped polygon file contains any polygons
      # if this is not the case:
      if (length(mask_dummy) == 0) {

        # copy one band of the current tile
        mask <- img[[1]]
        # set all values of the band to 0 (no reference polygon)
        values(mask) <- 0

        # if it is the case
      } else {

        # copy one band of the current tile
        mask2 <- img[[1]]
        # set all values of the band to 0
        values(mask2) <- 0
        # then rasterie the polygon objects into the
        # raster file
        mask <- rasterize(mask_dummy,mask2)
        mask[mask==1]<-1
        plot(mask)
      }

      # !!!!! set the output path for the image tiles
      setwd("C:/BSc_Arbeit/Twigs/Klassifikation/test_zweige/i")
      # compile an output filename
      imgname <- paste0("img", u, "_", i1, "_", i2, ".tif")
      # save the file to the harddisc
      writeRaster(img2, file=imgname)

      # !!!!! set the output path for the mask tiles
      setwd("C:/BSc_Arbeit/Twigs/Klassifikation/test_zweige/m")
      # compile an output filename
      maskname <- paste0("mask", u,"_", i1, "_", i2, ".tif")
      # save the file to the harddisc
      writeRaster(mask, file = maskname)

    }
    # print id of current iteration of current image-subset
    print(i1)
  }
}

